In [ ]:
using PyCall
using Statistics
using LinearAlgebra
using Random
;

In [59]:
# PyCallを用いてPythonのライブラリをインポート
np = pyimport("numpy")
@pyimport tensorflow 
@pyimport tf_quant_finance as tff

In [60]:
function generate_random_combinations(len, d, num_samples)
    Random.seed!(100)

    possible_values = collect(1:len)
    random_combinations = Vector{Int}[]
    
    for _ in 1:num_samples
        combination = rand(possible_values, d)
        push!(random_combinations, combination)
    end
    
    return random_combinations
end

generate_random_combinations (generic function with 1 method)

In [61]:
# オプション満期
T = 1.0
# 無リスク金利
r = 0.01
# strike
K = 100.0
# 原資産数
d = 5
# 原資産価格初期値
#S0 = [100.0] * d
# stock0 = collect(90.0:0.3:119.7)
stock0 = collect(70.0:1.3:198.7)
# ボラティリティ
#vols = [0.5] * d
vols = fill(0.5, d)
# 相関行列
corrMat = Matrix{Float64}(I, d, d)
for i in 1:d
    for j in 1:d
        if i != j
            #corrMat[i, j] = 1/3
            corrMat[i, j] = 0.3
        end
    end
end
cut = 100
localindex_v = [51 for _ in 1:d]
#localindex_v = [47, 16, 12, 36, 62, 97, 22, 100, 63, 76, 54]
#localindex_v = [cut for _ in 51:d]
localindex_s = [72, 74, 86, 63, 79]
#localindex_s = [80 for _ in 1:d]
nPath = 1000000

# パス数
#nPath = 10000000
σs = collect(0.15:0.001:0.249)
;

In [62]:
function calculate_option_price(T::Float64, r::Float64, K::Float64, d::Int64, stock0::Vector{Float64}, localindex_v::Vector{Int64}, localindex_s::Vector{Int64}, nPath::Int64, σs::Vector{Float64}, corrMat)::Tuple{Float64, Float64}
    vols = [σs[i] for i in localindex_v ]
    #vols = fill(0.2, d)
    S0 = fill(100, d)
    #S0 = [stock0[i] for i in localindex_s ]

    process = tff.models.MultivariateGeometricBrownianMotion(
        dim=d,
        means=fill(r, d),
        volatilities=vols,
        corr_matrix=corrMat)
    
    paths = process.sample_paths(
        times=[T],
        initial_state=S0,
        random_type=tff.math.random.RandomType.PSEUDO,
        num_samples=nPath).numpy()

    
    payoffs = max.(minimum(paths[:, 1, :], dims=2) .- K, 0.0)
    #payoffs = vec(payoffs)  # 配列をベクトルに変換
    mean_pv = exp(-r * T) * mean(payoffs)
    std_pv = exp(-r * T) * std(payoffs)
    #pv = exp(-r * T) * mean(max.(minimum(paths[:, 1, :], dims=2) .- K, 0.0))
    return (mean_pv, std_pv)
    return pv

end

calculate_option_price (generic function with 1 method)

In [63]:
# オプション価格の計算
mean_pv, std_pv = calculate_option_price(T, r, K, d, stock0, localindex_v, localindex_s, nPath, σs, corrMat)
println("Present Value: ", mean_pv)
println("Standard Deviation: ", std_pv)
#pv = calculate_option_price(T, r, K, d, S0, localindex, nPath, σs, corrMat)
#println(pv)
;

Present Value: 0.6813049243049988
Standard Deviation: 2.9354292639336634


In [64]:
true_value = 34.53644785048398
println("真の値: ", true_value)
;

真の値: 34.53644785048398


In [65]:
SE = std_pv/sqrt(nPath)
CI = 1.96 * std_pv/sqrt(nPath)
#@show 1.96 * SE
#println("絶対誤差の指標の代わり=", 2 * 1.96 * SE) 
println("CI: ", CI) 
println("Present Value + CI: ", mean_pv + CI) 
println("Present Value - CI: ", mean_pv - CI) 
@show mean_pv + CI #　信頼区間
@show mean_pv - CI # 信頼区間
;

CI: 0.00575344135730998
Present Value + CI: 0.6870583656623087
Present Value - CI: 0.6755514829476889
mean_pv + CI = 0.6870583656623087
mean_pv - CI = 0.6755514829476889


In [66]:
println("相対誤差の指標の代わり=CI / 真の値") 
rel = abs(mean_pv - true_value) / true_value
println("相対誤差: ", rel)
s_rel = CI / true_value
println("CI / 真の値: ", s_rel)
println("相対誤差 + CI/真の値: ", rel + s_rel)
;

相対誤差の指標の代わり=CI / 真の値
相対誤差: 0.980272872089958
CI / 真の値: 0.00016659042013289602
相対誤差 + CI/真の値: 0.9804394625100908


In [67]:
time_mc = @elapsed begin
   for i in 1:20
       mean_pv, std_pv = calculate_option_price(T, r, K, d, stock0, localindex_v, localindex_s, nPath, σs, corrMat)
   end
end

@show time_mc / 20

time_mc / 20 = 0.5847140975


0.5847140975

In [68]:
# pv = 1.3885828365963908

# sigma = 0.01879949670107938  #nPath = 1e5
# sigma = 0.005973520418390066 #nPath = 1e6

In [69]:
# pv = 0.1455493754265649

# sigma = 0.04713544576157798  #nPath = 1e5
# sigma = 0.01565396335528381  #nPath = 1e6

In [70]:
len = 100
num_samples = 100
combinations = generate_random_combinations(len, d, num_samples)
;

In [71]:
mean_pvs = []
CIs = []
;

In [72]:
for inds_ in combinations
    @show inds_
    mean_pv, std_pv = calculate_option_price(T, r, K, d, stock0, inds_, inds_, nPath, σs, corrMat)
    push!(mean_pvs, mean_pv)
    CI = 1.96 * std_pv/sqrt(nPath)
    push!(CIs, CI)
end

inds_ = [13, 18, 59, 92, 5]
inds_ = [29, 87, 85, 51, 3]
inds_ = [73, 51, 32, 35, 24]
inds_ = [59, 88, 18, 69, 38]
inds_ = [72, 29, 65, 19, 58]
inds_ = [74, 20, 86, 62, 45]
inds_ = [65, 42, 57, 13, 23]
inds_ = [26, 31, 39, 25, 97]
inds_ = [61, 91, 28, 23, 82]
inds_ = [36, 18, 53, 18, 39]
inds_ = [62, 14, 67, 6, 52]
inds_ = [53, 15, 85, 18, 81]
inds_ = [8, 35, 31, 32, 61]
inds_ = [93, 61, 49, 7, 35]
inds_ = [53, 49, 50, 64, 98]
inds_ = [84, 38, 73, 7, 18]
inds_ = [8, 97, 70, 45, 4]
inds_ = [99, 63, 27, 60, 7]
inds_ = [67, 93, 29, 5, 80]
inds_ = [33, 45, 50, 11, 65]
inds_ = [46, 46, 94, 18, 99]
inds_ = [65, 52, 44, 56, 72]
inds_ = [93, 54, 91, 30, 63]
inds_ = [62, 29, 45, 20, 13]
inds_ = [34, 35, 43, 97, 56]
inds_ = [69, 53, 1, 61, 40]
inds_ = [25, 43, 17, 81, 6]
inds_ = [67, 75, 60, 45, 33]
inds_ = [21, 15, 37, 43, 3]
inds_ = [74, 86, 70, 54, 68]
inds_ = [40, 74, 31, 17, 66]
inds_ = [81, 2, 14, 80, 38]
inds_ = [85, 4, 91, 57, 31]
inds_ = [92, 86, 78, 80, 22]
inds_ = [39, 83, 51, 21, 53]


In [73]:
maximum(CIs)

0.0063033018496348805

In [74]:
mean(CIs)

0.005570825622200468

In [75]:
mean_pvs

100-element Vector{Any}:
 0.6275836969957205
 0.6517823392877111
 0.6557419614407326
 0.6773587356238542
 0.6644162696113908
 0.6785760466864937
 0.6462729261888376
 0.6448003825622093
 0.6734568361733054
 0.6359023025668945
 ⋮
 0.6533193610211576
 0.6906000508122578
 0.6873980325308414
 0.6939130965667286
 0.6806595859049862
 0.6070501571253296
 0.6366623290327541
 0.6280623587049065
 0.6505165023971402